In [1]:
from datetime import datetime
from faker import Faker
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
social_media_users_df = pd.read_csv("/disk3/eric/dataset_graph/sample/genres.csv")

In [3]:
genres = set()

for _, row in tqdm(social_media_users_df[["Interests"]].iterrows(), total=len(social_media_users_df)):
    for genre in row["Interests"].replace("'", "").split(","):
        genre = genre.strip()
        genres.add(genre)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
fake = Faker()

In [5]:
def create_fake_person():
    birth_date = fake.date_of_birth()
    age = datetime.date(datetime.now()).year - birth_date.year
    
    bot_score = fake.pyfloat(left_digits=0, right_digits=2, positive=True, min_value=None, max_value=None)
  
    _, _, city, country, state = fake.local_latlng(country_code="US")
    state = state.split("/")[-1].replace("_", " ")
    location = " ".join([city, state, country])
  
    return [fake.uuid4(), fake.first_name(), fake.last_name(), birth_date.strftime("%Y-%m-%d"), age, location, fake.random_element(genres), bot_score]

In [ ]:
people = [create_fake_person() for _ in range(1000)]

In [7]:
def create_fake_poster():
    birth_date = fake.date_of_birth()
    age = datetime.date(datetime.now()).year - birth_date.year
    
    return [fake.uuid4(), fake.user_name(), age, ";".join([fake.random_element(genres) for _ in range(fake.random_int(min=1, max=5))])]

In [ ]:
posters = [create_fake_poster() for _ in range(200)]

In [9]:
followers = []

for person in people:
    followers.extend([[person[0], fake.random_element(people)[0]] for _ in range(fake.random_int(min=2, max=10))])

In [10]:
subscribers = []

for person in people:
    subscribers.extend([[person[0], fake.random_element(posters)[0]] for _ in range(fake.random_int(min=2, max=10))])

In [11]:
people_df = pd.DataFrame(people, columns=["personId:ID(Person-ID)", "first_name", "last_name", "birth_date:date", "age:int", "location", "genre", "bot_score:float"])
people_df[":LABEL"] = "person"
people_df.head()

,personId:ID(Person-ID),first_name,last_name,birth_date:date,age:int,location,genre,bot_score:float,:LABEL
0,667e4f00-f5e7-4a5e-94a3-ce27256a186c,Jeremiah,West,1994-08-15,30,Martinsburg New York US,Art,0.28,person
1,417812f9-aea7-4070-9eaa-d614e9d8c7f7,Joseph,Reyes,1948-08-18,76,Yonkers New York US,Outdoor activities,0.98,person
2,b89835dc-2186-4881-9599-8a45567ca320,Anne,Holmes,1988-10-16,36,Sicklerville New York US,Cooking,0.85,person
3,f352f169-7bf5-4608-a254-85e55137ec84,Barbara,Hoffman,2011-09-12,13,Riverbank Los Angeles US,Cooking,0.35,person
4,ab58c5fe-8863-49e4-a23e-95c1871f6f3a,Charles,Carroll,1983-04-01,41,Nampa Boise US,History,0.46,person


In [12]:
posters_df = pd.DataFrame(posters, columns=["posterId:ID(Poster-ID)", "username", "age:int", "genres:string[]"])
posters_df[":LABEL"] = "poster"
posters_df.head()

,posterId:ID(Poster-ID),username,age:int,genres:string[],:LABEL
0,2ccbfa41-c6e7-430d-a80d-3b9cd1d0c5c0,michellegonzales,69,Movies;Fashion;Fitness,poster
1,778f82c9-66ee-4040-8600-c01d04e3ed79,jennaclark,45,Food and dining,poster
2,818edb86-43b6-4152-b842-8b69d20c97f3,william08,33,Education and learning;Education and learning,poster
3,8f1ed238-6a1a-478c-85ac-4f942e25a975,michaelperez,2,Travel;Politics,poster
4,f244c84a-6527-498b-856f-f04667d3d5e3,vlucas,40,Food and dining;Pets;Business and entrepreneur...,poster


In [13]:
followers_df = pd.DataFrame(followers, columns=["personId:START_ID(Person-ID)", "personId:END_ID(Person-ID)"])
followers_df[":TYPE"] = "follows"
followers_df.head()

,personId:START_ID(Person-ID),personId:END_ID(Person-ID),:TYPE
0,667e4f00-f5e7-4a5e-94a3-ce27256a186c,95642ccc-dba7-48cb-80bd-4b181669499e,follows
1,667e4f00-f5e7-4a5e-94a3-ce27256a186c,7b6e3bd4-4534-48de-94ea-7c9bf09fd88d,follows
2,667e4f00-f5e7-4a5e-94a3-ce27256a186c,47b202ff-fa33-4c47-a626-5a5ce055e604,follows
3,667e4f00-f5e7-4a5e-94a3-ce27256a186c,e9e110e3-c16e-4d9b-8739-f603896bc2d3,follows
4,417812f9-aea7-4070-9eaa-d614e9d8c7f7,88fa686d-2d38-4a88-8a6e-c18a4bd06694,follows


In [14]:
subscribers_df = pd.DataFrame(subscribers, columns=["personId:START_ID(Person-ID)", "posterId:END_ID(Poster-ID)"])
subscribers_df[":TYPE"] = "subscribes_to"
subscribers_df.head()

,personId:START_ID(Person-ID),posterId:END_ID(Poster-ID),:TYPE
0,667e4f00-f5e7-4a5e-94a3-ce27256a186c,a3279e3f-e979-474d-8972-20b3d899147d,subscribes_to
1,667e4f00-f5e7-4a5e-94a3-ce27256a186c,74221659-10da-4530-8a06-12ce1c967fda,subscribes_to
2,667e4f00-f5e7-4a5e-94a3-ce27256a186c,8494050f-2e79-4767-b8f1-af56ba1a3040,subscribes_to
3,667e4f00-f5e7-4a5e-94a3-ce27256a186c,341845fc-7042-488f-8d14-c1b6f37b867f,subscribes_to
4,667e4f00-f5e7-4a5e-94a3-ce27256a186c,f0b40d89-6491-4a86-bf35-2dba31de230a,subscribes_to


In [15]:
people_df.to_csv("/disk3/eric/dataset_graph/sample/nodes/people.csv", index=False)
posters_df.to_csv("/disk3/eric/dataset_graph/sample/nodes/posters.csv", index=False)
followers_df.to_csv("/disk3/eric/dataset_graph/sample/edges/follows.csv", index=False)
subscribers_df.to_csv("/disk3/eric/dataset_graph/sample/edges/subscribes.csv", index=False)